https://www.philschmid.de/fine-tune-llms-in-2024-with-trl

``` bash
pip install sentencepiece transformers trl datasets pandas numpy protobuf accelerate bitsandbytes flash-attn
pip install torch==2.0.1
```
На Selectel pytroch должен быть: '2.0.1+cu117

In [1]:
import torch
print(f"Версия pytorch: {torch.__version__}")

# pip install torch==2.0.1

if torch.cuda.is_available():
    print('CUDA is available')
    print(torch.cuda.get_device_properties(0))
elif torch.backends.mps.is_available():
    print('Apple MPS is available')
else:
    print('No GPU available')
# _CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40384MB, multi_processor_count=108)

Версия pytorch: 2.0.1+cu117
CUDA is available
_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40384MB, multi_processor_count=108)


In [12]:
import os
import numpy as np
import pandas as pd
import torch
# from tqdm import tqdm
from datasets import(
  Dataset,
  load_dataset,
  load_dataset_builder
)
from peft import (
  LoraConfig,
  get_peft_model,
  PeftConfig,
  PeftModel
)
from datasets import load_dataset
from transformers import (
  AutoModelForCausalLM,
  AutoTokenizer,
  BitsAndBytesConfig,
  TrainingArguments,
  AutoConfig,
  pipeline,
  GenerationConfig,
  DataCollatorForLanguageModeling
)
from trl import SFTTrainer
from huggingface_hub import login
pd.set_option('max_colwidth', 400)
from dotenv import load_dotenv
load_dotenv()

login(
  token=os.getenv("HF_TOKEN"),
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/aweeu/.cache/huggingface/token
Login successful


# Пример использования промпта

In [7]:
model_id="IlyaGusev/saiga2_13b_lora" # pip install sentencepiece
# model_id="Open-Orca/Mistral-7B-OpenOrca" # pip install sentencepiece


tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False
)

chat = [
   {"role": "system", "content": "You're AI assistant"},
   {"role": "user", "content": "Hello, how are you?"},
   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
   {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

print(tokenizer.apply_chat_template(chat, tokenize=False))

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565

No chat template is defined for this tokenizer - using the default template for the LlamaTokenizer class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



<s>[INST] <<SYS>>
You're AI assistant
<</SYS>>

Hello, how are you? [/INST] I'm doing great. How can I help you today? </s><s>[INST] I'd like to show off how chat templating works! [/INST]


# Подготовка датасета

Создание датасета описано в другом файле, где происходила выгрузка чатов

In [5]:
train_dataset = load_dataset(
    path="./dataset_2/dataset_short", # указываем путь до папки с файлами
    # data_files="test_dataset.json", # или можем указать прямой путь до файла
    split="train" # указываем что загружаем (test или train)
)
eval_dataset = load_dataset(
    path="./dataset_2/dataset_short", # указываем путь до папки с файлами
    # data_files="test_dataset.json", # или можем указать прямой путь до файла
    split="test" # указываем что загружаем (test или train)
)

In [10]:
# print(tokenizer.apply_chat_template(train_dataset['messages'][1], tokenize=False))

# Finetuning

## Загрузка модели

### Mistral-7B-OpenOrca

In [8]:
model_id="Open-Orca/Mistral-7B-OpenOrca" # модель, которую будем дообучать
lora_checkpoints="./lora-checkpoints" # директория с LoRA-адаптерами, которые получим на выходе

def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        # use_fast=True,
        # turst_remote_code=True
    )
    tokenizer.padding_side = 'right' # to prevent warnings
    tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=False, # Загружает модель в 4-битном формате для уменьшения использования памяти.
    #     load_in_8bit=True,
    #     bnb_4bit_quant_type="nf4", # Указывает тип квантования, в данном случае "nf4" (nf4/dfq/qat/ptq/fp4)
    #     bnb_4bit_compute_dtype="float16", # Устанавливает тип данных для вычислений в 4-битном формате как float16.
    #     bnb_4bit_use_double_quant=False # Указывает, что не используется двойное квантование.
    # )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # config=AutoConfig.from_pretrained(model_id)
        torch_dtype=torch.bfloat16, # torch.float16
        # quantization_config=bnb_config,
        device_map="auto",
        # attn_implementation="flash_attention_2"
    )

    model.config.use_cache=False # Отключает кэширование внутренних состояний модели во время генерации текста. Это может быть полезно для экономии памяти, особенно при работе с длинными последовательностями.
    model.config.pretraining_tp=1 # параметр, связанный с техниками распределенного обучения 
    
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.69s/it]


### Llama2-13b-hf + Saiga

In [7]:
from peft import PeftConfig, PeftModel
from transformers import AutoTokenizer

lora_checkpoints="./lora-checkpoints"

lora_adapter = "IlyaGusev/saiga2_13b_lora"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

model = AutoModelForCausalLM.from_pretrained(
            base_model,
            # load_in_8bit=True,
            # load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map="cuda",
            # attn_implementation="flash_attention_2"
        )
model = PeftModel.from_pretrained(
            model,
            lora_adapter,
            torch_dtype=torch.float16,
            device_map="cuda"
        )

model = model.merge_and_unload()

Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.57s/it]


### Llama2-13b-hf

In [ ]:
model_id="meta-llama/Llama-2-13b-hf" # модель, которую будем дообучать
lora_checkpoints="./lora-checkpoints" # директория с LoRA-адаптерами, которые получим на выходе

def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        # use_fast=True,
        # turst_remote_code=True
    )
    tokenizer.padding_side = 'right' # to prevent warnings
    tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=False, # Загружает модель в 4-битном формате для уменьшения использования памяти.
    #     load_in_8bit=True,
    #     bnb_4bit_quant_type="nf4", # Указывает тип квантования, в данном случае "nf4" (nf4/dfq/qat/ptq/fp4)
    #     bnb_4bit_compute_dtype="float16", # Устанавливает тип данных для вычислений в 4-битном формате как float16.
    #     bnb_4bit_use_double_quant=False # Указывает, что не используется двойное квантование.
    # )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # config=AutoConfig.from_pretrained(model_id)
        torch_dtype=torch.bfloat16, # torch.float16
        # quantization_config=bnb_config,
        device_map="auto",
        # attn_implementation="flash_attention_2"
    )

    model.config.use_cache=False
    model.config.pretraining_tp=1
    
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

### Params check

In [ ]:
# Check there're no parameters onto CPU
for n, p in model.named_parameters():
    if p.device.type == 'meta':
        print(f"{n} is on meta!")

print(model.config.max_position_embeddings)
print(model.config.eos_token_id)

## Настройка параметров обучения

In [ ]:
# Оценка количества параметров в модели для обучения
def print_trainable_parameters(model):
    trainable_params = 0
    non_trainable_params = 0
    all_params = 0

    print("Trainable parameters:")
    for name, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            print(f"    {name}")
        else:
            non_trainable_params += param.numel()

    print("\nNon-Trainable Parameters:")
    for name, param in model.named_parameters():
        if not param.requires_grad():
            print(f"    {name}")

    print(
        f"\nSummary:\n  Trainable params: {trainable_params}\n  Non-Trainable params: {non_trainable_params}\n  All params: {all_params}"
    )

print_trainable_parameters(model)

In [ ]:
print(model)

In [8]:
# НАСТРОИТЬ!!!

# from peft import prepare_model_for_kbit_training
# model = prepare_model_for_kbit_training(model) # только для QLoRA

peft_config = LoraConfig(
        r=8, # определяет ранг матриц адаптации в LoRA. Этот параметр контролирует количество параметров адаптации, которые добавляются к модели. Более низкий ранг означает меньше дополнительных параметров и, следовательно, меньшее влияние на исходные веса модели.
        lora_alpha=32, # масштабирующий коэффициент для матриц адаптации. Этот коэффициент управляет степенью, с которой адаптированные веса влияют на поведение модели.
        lora_dropout=0.05,
        # target_modules=[
        #     "q_proj",
        #     "o_proj",
        #     "k_proj",
        #     "v_proj"
        # ]
        bias="none", # указывает, должен ли добавляться смещение (bias) к параметрам LoRA
        task_type="CAUSAL_LM", # "CAUSAL_LM" означает причинно-следственное (каузальное) языковое моделирование ????????????
        # target_modules="all-linear"
    )

training_arguments = TrainingArguments(
        output_dir=lora_checkpoints, # Путь к каталогу, где будут сохраняться обученная модель и другие выходные данные.
        per_device_train_batch_size=4, # Размер пакета (batch size) для обучения на каждом устройстве. Определяет количество образцов данных, обрабатываемых за один шаг обучения на каждом устройстве.
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=1, # Количество шагов накопления градиента. Это позволяет эффективно увеличить размер пакета, не увеличивая использование памяти.
        gradient_checkpointing=True, # чекпоинты градиентов для сохранения памяти
        weight_decay=0.001,
        optim="paged_adamw_32bit", # 8-16-32
        learning_rate=1e-4,
        lr_scheduler_type="cosine", # "constant"
        # fp16=False,
        # bf16=False,
        # tf32=True,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        save_strategy="steps", # Стратегия сохранения модели. "epoch" означает, что модель будет сохраняться после каждой эпохи обучения.
        logging_steps=100, # Количество шагов обучения между логированием метрик обучения.
        num_train_epochs=1, # 1-2-3
        # eval_steps=50,
        save_steps=500,
        max_steps=-1, # Максимальное количество шагов обучения. Обучение закончится, когда будет достигнуто это число шагов, даже если не все эпохи были завершены.
        # report_to="tensorboard",
        # push_to_hub=True
    )

trainer = SFTTrainer(
        model=model,
        peft_config=peft_config,
        args=training_arguments,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        # dataset_text_field="messages", # не передаем это поле, если датасет уже был предобработан в нужный формат
        # dataset_kwargs={
        #     "add_special_tokens": False,  # We template with special tokens
        #     "append_concat_token": False, # No need to add additional separator token
        # },
        neftune_noise_alpha=5,
        # packing=True,
        tokenizer=tokenizer,
        max_seq_length=512,
        # neftune_noise_alpha=5
    )

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:234: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(
Map:   0%|          | 0/115 [00:00<?, ? examples/s]
No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

Map: 100%|██████████| 29/29 [00:00<00:00, 1899.15 examples/s]


In [ ]:
model.config.use_cache = False

In [9]:
# trainer.train(resume_from_checkpoint=True)
trainer.train()

# save model
trainer.save_model("./lora_adapters")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


# Объединение модели с LoRA-адаптером

In [2]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import torch

model_id="Open-Orca/Mistral-7B-OpenOrca"
lora_adapters = "./lora_adapters"
merged_model_path = "./merged-model"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = PeftModel.from_pretrained(model, lora_adapters)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Merge LoRA and base model and save
model = model.merge_and_unload()
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token

# model.save_pretrained(merged_model_path, safe_serialization=True, max_shard_size="2GB")
# tokenizer.save_pretrained(merged_model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.80s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Загрузка на HF

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# !huggingface-cli login
hf_repo_name = "aesedeu/openorca-7b-stlt-chatbot"

model.push_to_hub(
    hf_repo_name,
    check_pr=True,
    max_shard_size="5GB"
)
tokenizer.push_to_hub(
    hf_repo_name,
    check_pr=True
)

README.md: 100%|██████████| 28.0/28.0 [00:00<00:00, 110kB/s]
model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]




model-00001-of-00003.safetensors:   0%|          | 16.4k/4.94G [00:00<23:26:25, 58.6kB/s]

model-00001-of-00003.safetensors:   0%|          | 2.34M/4.94G [00:00<11:28, 7.17MB/s]   

model-00001-of-00003.safetensors:   0%|          | 5.88M/4.94G [00:00<05:11, 15.9MB/s]

model-00001-of-00003.safetensors:   0%|          | 13.3M/4.94G [00:00<03:36, 22.7MB/s]

model-00001-of-00003.safetensors:   0%|          | 16.0M/4.94G [00:01<06:13, 13.2MB/s]

model-00001-of-00003.safetensors:   1%|          | 30.8M/4.94G [00:01<03:46, 21.6MB/s]

model-00001-of-00003.safetensors:   1%|          | 33.4M/4.94G [00:02<06:00, 13.6MB/s]

model-00001-of-00003.safetensors:   1%|          | 43.2M/4.94G [00:02<04:35, 17.8MB/s]

model-00001-of-00003.safetensors:   1%|          | 46.7M/4.94G [00:02<04:14, 19.3MB/s]

model-00001-of-00003.safetensors:   1%|          | 59.8M/4.

CommitInfo(commit_url='https://huggingface.co/aesedeu/openorca-7b-stlt-chatbot/commit/29b1ef30c812b79fb48d66f9eb17462091c9a702', commit_message='Upload tokenizer', commit_description='', oid='29b1ef30c812b79fb48d66f9eb17462091c9a702', pr_url=None, pr_revision=None, pr_num=None)

# Инференс обученной модели

## Загрузка модели

### модель + адаптеры

In [4]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

model_id="Open-Orca/Mistral-7B-OpenOrca"
lora_adapters = "./lora_adapters"

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(
  model,
  lora_adapters,
  torch_dtype=torch.float16,
  device_map="cuda"
)

model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

### Загрузка через LoRA-адаптеры

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

lora_adapters = "./lora_adapters"
model_id="Open-Orca/Mistral-7B-OpenOrca"

model = AutoPeftModelForCausalLM.from_pretrained(
  lora_adapters,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token

/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/3_model_mistralorca/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.28s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been a

### Загрузка объединенной модели

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

merged_model_path = "./merged-model"

model = AutoModelForCausalLM.from_pretrained(
  merged_model_path,
  device_map="cuda",
  # load_in_8bit=True,
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token

## Инференс вручную

In [4]:
model_id="../Mistral-7B-OpenOrca" 

# Настройка токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
SYSTEM_PROMPT = f"""Ты - русскоязычный ассистент Степан. Отвечаешь только на вопросы о лотереях."""
# SYSTEM_PROMPT = f"""Ты - пьяный пират, который ищет свой корабль. Разговаривай как пьяный пират."""

QUESTION = 'Как вернуть билет?'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'
input_message = input_message[:-5].strip() + "\n"
input_message

'<s>system\nТы - русскоязычный ассистент Степан. Отвечаешь только на вопросы о лотереях.</s>\n<s>user\nКак вернуть билет?</s>\n<s>assistant\n'

In [13]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 128,
    repetition_penalty=1.3,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)
print(output_message)

В случаи возврата бонсов без выплаты по бонусной игры, список участников и результат розагружается отдельно в раздела «Все участия» не ранее чем через двадцати четверги после завершения турнира.  Для проведения технического обслуживания клиентов можем быть требуться короткие передышки из делательности примерно минут. Мы обращаем ваше внимание на то, что способ получениябо


## Инференс с помощью pipelines

In [15]:
from transformers import pipeline
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [16]:
from datasets import load_dataset
from random import randint

eval_dataset = load_dataset(
    path="./dataset",
    # data_files="test_dataset.json", # убираем если нужно загрузить train+test
    split="test" # убираем если нужно загрузить train+test
)

rand_idx = randint(0, len(eval_dataset))

In [19]:
# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)

outputs = pipe(
    prompt,
    repetition_penalty=1.1,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    # top_k=40,
    top_p=0.9,
    eos_token_id=pipe.tokenizer.eos_token_id,
    pad_token_id=pipe.tokenizer.pad_token_id,
    # no_repeat_ngram_size=15,
    # bos_token_id=1,
    # eos_token_id=2
)

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print("==================================")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print("==================================")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
Здравствуйте Как проверить билет В телевизоре повторно покажут сегодняшний тираж Хорошо Ну что
Original Answer:
  Для проверки билета, пожалуйста, перейдите по ссылке https://www.stoloto/check-ticket
Generated Answer:
Для проверки билета, пожалуйста, перейдите по ссылке https://www.stoloto/check-ticket  Если при покупке билета Вы не указывали номер телефона, то номер тиража и билета указаны на самом билете. Ознакомиться с инструкцией, как принять участие в лотерее на сайте, Вам не нужно (https://www.stoloto/lottery/onlinesale?from=header)   Для проверки билета, пожалуйста, перейдите по ссылке https://www.stoloto/check-ticket  Если при покупке билета Вы не указывали номер телефона, то номер тиража и билета указаны на самом билете. Ознакомиться с инструкцией, как принять участие в лотерее на сайте, Вам не нужно (https://www.stoloto


https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/fine-tune-llms-in-2024-with-trl.ipynb